<a href="https://colab.research.google.com/github/arkwave/COVID-19/blob/master/ImageNet_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [6]:
from utils import get_dataset, PairGenerator, test_embedding, visualize_deformation, plot_losses
from networks import pairwise_embedding_encoder, get_embedding_model, pairwise_embedding_network, get_encoding_model
from losses import contrastive_loss, reconstruction_loss
from training import train_network
from tensorflow.keras.utils import plot_model
from keras.optimizers import Adam
import numpy as np 
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [ ]:
## define training params. 
margin = 1
p_norm = 2 
batch_size = 32

## specify model parameters and losses. 
model_type = 'embenc'
optimizer = Adam(learning_rate=0.01)
activation = 'sigmoid'
num_neurons = 300
output_dim = 2
input_dim = (1, trainX.shape[-1])
loss1 = lambda y_true, y_pred: contrastive_loss(y_true, y_pred, margin=margin, norm=p_norm)
loss2 = lambda y_true, y_pred: reconstruction_loss(y_true, y_pred)
losses = {'reconstruction': loss2,
          'embedding': loss1}
epochs = 5
max_iters = 10000

In [ ]:
## create generator
tripgen = PairGenerator(trainX, trainY)

## create the model. 
if model_type == 'embenc':
    model = pairwise_embedding_encoder(input_dim, output_dim, num_neurons=num_neurons,
                                       losses=losses, optimizer=optimizer, f=activation,
                                       mu=0.75, autoencoding_layers=[100])
elif model_type == 'siamese':
    model = pairwise_embedding_network(input_dim, output_dim, num_neurons, loss1, optimizer, f=activation)

## visualize model topology. 
plot_model(model, 'full_model.png', expand_nested=True, show_shapes=True)
plot_model(model.get_layer('encodingnet'), 'encodingnet.png', expand_nested=True, show_shapes=True)
plot_model(model.get_layer('embeddingnet'), 'embeddingnet.png', expand_nested=True, show_shapes=True)

In [ ]:
## train network 
model, losses = train_network(model, tripgen, batch_size, max_iters, type_=model_type)

In [ ]:
# ## visualize the embeddings. 
embedding_model = get_embedding_model(model, input_dim=input_dim)
embedding = embedding_model.predict(trainX.reshape(-1, 1, input_dim[-1]))
embedding = embedding.reshape(embedding.shape[0], embedding.shape[2])
embedding_fig = test_embedding(trainX, trainY, embedding)
# # original_fig, embedded_fig_no_noise, embedding_fig_noise, vector_field = visualize_deformation(embedding_model, input_dim)    
# encoding_model = get_encoding_model(model, input_dim=input_dim)

In [ ]:
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import pandas as pd 
img_w, img_h = 28,28
zoom = 0.5

# subsample a section of both dataframes in a stratified fashion. 
inds = [] 
for cl in set(trainY):
    cl_inds = np.random.choice(np.where(trainY == cl)[0],
                               500,
                               replace=False)
    inds.extend(cl_inds)

plt_trainX = pd.DataFrame(trainX[inds, :]).reset_index(drop=True)
plt_embedding = embedding[inds, :]
print(plt_embedding.shape)

fig, ax = plt.subplots(figsize=(24,16))

for i, row in plt_trainX.iterrows():
    image = row.values.reshape((img_w, img_h))
    im = OffsetImage(image, zoom=0.3)
    ab = AnnotationBbox(im, (plt_embedding[i, 0], plt_embedding[i, 1]), xycoords='data', frameon=False)
    ax.add_artist(ab)
    ax.update_datalim([(embedding[i, 0], embedding[i, 1])])
    ax.autoscale()